In [13]:
import numpy as np

In [ ]:
class environment:
    def __init__(self, grid_height, grid_width):
        """
        The map is initialized with height and width are variable of your choice
        start: List of location where you step in, you get to the corresponding location in list 'end'
                For example:
                    if you step in location start[3] then you get to new location end[3] then obtain the reward value of reward[3]
        reward: List of reward value where you move from a location in 'start' list to the corresponding location in 'end' list
        """
        self.height = grid_height
        self.width = grid_width
        self.start = []
        self.end = []
        self.reward = []
        self.map = np.array([i for i in range(grid_height * grid_width)])
        self.action_space = [0, 1, 2, 3]

    def get_map(self):
        print(self.map.reshape([self.width, self.height]))

    def get_num_state(self):
        return self.height * self.width

    def map_Designate(self, start_cell, end_cell, reward):
        self.start.append(start_cell)
        self.end.append(end_cell)
        self.reward.append(reward)

    def get_Observation(self, location, action):
        # Default reward = 0
        reward = 0
        new_location = 0
        # Action: UP: 0, DOWN: 1, LEFT: 2, RIGHT: 3
        # Actions that get the agent out of the map, result in no change at all
        if action == 0:  # UP
            if location - self.width < 0:
                new_location = location
            else:
                new_location = location - self.width

        elif action == 1:  # DOWN
            if location + self.width > self.height * self.width - 1:
                new_location = location
            else:
                new_location = location + self.width

        elif action == 2:  # LEFT
            if location % self.width == 0:
                new_location = location
            else:
                new_location = location - 1

        elif action == 3:  # RIGHT
            if (location + 1) % self.width == 0:
                new_location = location
            else:
                new_location = location + 1

        # If the agent is at special locations, immediately moves to corresponding destinations, gain reward
        if new_location in self.start:
            idx = self.start.index(new_location)
            new_location = self.end[idx]
            reward = self.reward[idx]

        return new_location, self.action_space, reward


class MABAgent:
    def __init__(self, envir, init_location):
        # Trace the reward
        self.reward_trace = []
        # initialize the first location
        self.location_now = init_location
        # TODO: implement other features to the agent so it can perform MAB algorithm
        self.last_action = None
        self.last_state = None
        self.value_table = {}  # format: {state: {action: [value, count]}}
        for state in range(envir.width * envir.height):
            self.value_table[state] = {}
            for action in envir.action_space:
                self.value_table[state][action] = [0, 0]

    def get_total_reward(self):
        return np.sum(self.reward_trace)


In [15]:
# Create environment
Envir = environment(8,8)
Envir.map_Designate(17,56,-15)
Envir.map_Designate(18,56,-15)
Envir.map_Designate(19,56,-15)
Envir.map_Designate(21,56,-15)
Envir.map_Designate(25,56,-15)
Envir.map_Designate(33,56,-15)
Envir.map_Designate(41,56,-15)
Envir.map_Designate(42,56,-15)
Envir.map_Designate(43,56,-15)
Envir.map_Designate(46,56,-15)
Envir.map_Designate(47,56,-15)
Envir.map_Designate(47,56,-15)
Envir.map_Designate(15,56,+15)
Envir.map_Designate(1,10,+5)
Envir.map_Designate(26,56,+20)

Envir.get_map()

[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]
